In [ ]:
#tool1
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

wiki_wrapper=WikipediaAPIWrapper(top_k_results=2,doc_content_chars_max=500)
wiki=WikipediaQueryRun(api_wrapper=wiki_wrapper)

In [ ]:
#tool2
from langchain_community.tools import ArxivQueryRun
from langchain_community.utilities import ArxivAPIWrapper

arx_wrapper=ArxivAPIWrapper(top_k_results=3,doc_content_chars_max=500)
arxiv=ArxivQueryRun(api_wrapper=arx_wrapper)

In [ ]:
arxiv.invoke("Attention is all you need")

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")
os.environ["TAVILY_API_KEY"]=os.getenv("TAVILY_API_KEY")

In [ ]:
#Tavili Search Tool(basically for google search)
from langchain_community.tools.tavily_search import TavilySearchResults

tavily=TavilySearchResults()

In [ ]:
tavily.invoke("Give me latest tech news")

In [ ]:
#combining all tools in a list
all_tools=[arxiv,wiki,tavily]

In [ ]:
#initialize the LLM Model
from langchain_groq import ChatGroq
llm=ChatGroq(model="deepseek-r1-distill-llama-70b")

In [ ]:
llm.invoke("What is llm?")

In [ ]:
#combining tools with llm
llm_and_tools=llm.bind_tools(tools=all_tools)

In [ ]:
llm_and_tools.invoke("What is the recent movie in india?")

In [ ]:
#Workflow
from typing_extensions import TypedDict
from langchain_core.messages import AnyMessage #Human/AI Message
from typing import Annotated #labelling
from langgraph.graph.message import add_messages #reducers in langgraph

In [ ]:
#State Schema
class State(TypedDict):
    messages:Annotated[list[AnyMessage],add_messages] #appends Human and AI messages without overriding messages

In [ ]:
#Entire Chatbot With LangGraph
from IPython.display import Image,display
from langgraph.graph import StateGraph,START,END
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition
 

In [ ]:
#Creating Nodes
def tool_calling_llm(state:State):
    return{"messages":[llm_and_tools.invoke(state["messages"])]} #which tool is invoked is sent in the value

#adding the nodes 
graph=StateGraph(State)
graph.add_node("tool_calling_llm", tool_calling_llm)
graph.add_node("tools",ToolNode(tools=all_tools))


In [ ]:
#adding the edges
graph.add_edge(START,"tool_calling_llm")
graph.add_conditional_edges(
    "tool_calling_llm",
    #if tool is called routes to tools
    #if tool not called , routes to END
    tools_condition,    
)
graph.add_edge("tools",END)


In [ ]:
#Compile the graph
graph_builder=graph.compile()

In [ ]:
#view
display(Image(graph_builder.get_graph().draw_mermaid_png()))

In [ ]:
messages=graph_builder.invoke({"messages":"1706.03762"})
for m in messages['messages']:
    m.pretty_print()

In [ ]:
messages=graph_builder.invoke({"messages":"Hey! I am Harnisha"})
for m in messages['messages']:
    m.pretty_print()